In [1]:
# Importing libraries
import pandas as pd
from pathlib import Path
import csv
from config import api_key
import requests
from pprint import pprint
import random

import plotly.express as px
from dash import Dash, dcc, html, Input, Output
import plotly.io as pio
pio.renderers.default = "chrome"

In [2]:
csv_path = Path("../Resources/global air pollution dataset.csv")
global_df = pd.read_csv(csv_path)
global_df.head()

,Country,City,AQI Value,AQI Category,CO AQI Value,CO AQI Category,Ozone AQI Value,Ozone AQI Category,NO2 AQI Value,NO2 AQI Category,PM2.5 AQI Value,PM2.5 AQI Category
0,Russian Federation,Praskoveya,51,Moderate,1,Good,36,Good,0,Good,51,Moderate
1,Brazil,Presidente Dutra,41,Good,1,Good,5,Good,1,Good,41,Good
2,Italy,Priolo Gargallo,66,Moderate,1,Good,39,Good,2,Good,66,Moderate
3,Poland,Przasnysz,34,Good,1,Good,34,Good,0,Good,20,Good
4,France,Punaauia,22,Good,0,Good,22,Good,0,Good,6,Good


In [3]:
global_df.columns

Index(['Country', 'City', 'AQI Value', 'AQI Category', 'CO AQI Value',
       'CO AQI Category', 'Ozone AQI Value', 'Ozone AQI Category',
       'NO2 AQI Value', 'NO2 AQI Category', 'PM2.5 AQI Value',
       'PM2.5 AQI Category'],
      dtype='object')

In [4]:
NorthAmerica_df = global_df.loc[global_df['Country'].isin(['United States of America', 'Canada', 'Mexico'])]
NorthAmerica_df.head()
NorthAmerica_df.count()

Country               3601
City                  3601
AQI Value             3601
AQI Category          3601
CO AQI Value          3601
CO AQI Category       3601
Ozone AQI Value       3601
Ozone AQI Category    3601
NO2 AQI Value         3601
NO2 AQI Category      3601
PM2.5 AQI Value       3601
PM2.5 AQI Category    3601
dtype: int64

In [5]:
unique_cities = NorthAmerica_df[['Country', 'City']].drop_duplicates().reset_index(drop=True)
cities_list = unique_cities['City']
cities_list.to_csv('../Resources/cities_list.csv')
unique_cities.to_csv('../Resources/citiesandcountry_list.csv')

# Making a list of cities. This is the entire list of 3601 countries 
list_cities = cities_list.to_list()

# Randomized list of 500 cities. This is probably better for plotting and to avoid hitting API limit
random_cities = random.sample(list_cities,500) # change to 500 or any number you want

# Shortened list of cities. Use this for testing code to avoid hitting API limit
short_cities = cities_list.head(3).tolist()


In [6]:
random_cities

['The Colony',
 'Salvatierra',
 'Morton Grove',
 'Wilmette',
 'Blacksburg',
 'Gulfport',
 'Murfreesboro',
 'Prairie Village',
 'Culpeper',
 'Justice',
 'Walla Walla',
 'Danbury',
 'Salaberry De Valleyfield',
 'San Luis De La Paz',
 'Paracho',
 'Uruapan',
 'El Dorado',
 'Coral Gables',
 'Pleasant Prairie',
 'Kokomo',
 'Hampton',
 'Campbellsville',
 'Guelph',
 'Riverton',
 'Foxborough',
 'Saint Petersburg',
 'Sierra Vista',
 'Morelia',
 'Red Deer',
 'Marshall',
 'Point Pleasant',
 'Selma',
 'Desoto',
 'Kaukauna',
 'Sedalia',
 'Leisure City',
 'Temple',
 'Roanoke Rapids',
 'Merrillville',
 'Hickory Hills',
 'Canandaigua',
 'Laredo',
 'Somers',
 'Gardendale',
 'West Orange',
 'Rosemont',
 'Kennesaw',
 'Kirksville',
 'Greenwood',
 'Hobbs',
 'Jacksonville',
 'Pichucalco',
 'Glen Carbon',
 'Joppatowne',
 'Apple Valley',
 'Nueva Rosita',
 'Pottstown',
 'Ottumwa',
 'Lighthouse Point',
 'Tuxtepec',
 'Maple Shade',
 'Actopan',
 'Effingham',
 'Evansville',
 'Rockwall',
 'Seymour',
 'Mashpee',
 'Le

In [7]:
# Acessing the open weather API to get lattitude and longitude values

url = "http://api.openweathermap.org/data/2.5/weather?"
units = "metric"

# Build partial query URL
query_url = f"{url}appid={api_key}&units={units}&q="

# Making API calls
lat = []
lon = []

# loop through list of cities
for city in random_cities:
    #print (city)
    
    response = requests.get(query_url + city).json()
    try:
        #print (response)
        lat.append(response['coord']['lat'])
        lon.append(response['coord']['lon'])
    except Exception as e:
        print(f'city not found or {e}')
        pass

city not found or 'coord'
city not found or 'coord'
city not found or 'coord'
city not found or 'coord'
city not found or 'coord'
city not found or 'coord'
city not found or 'coord'
city not found or 'coord'
city not found or 'coord'
city not found or 'coord'
city not found or 'coord'
city not found or 'coord'


In [8]:
# Accessing the open weather API to get air pollution values

air_url = 'http://api.openweathermap.org/data/2.5/air_pollution?' # current air pollution values

coord = [] # latitude and longitude coordinates
aqi = [] # air quality index
co = [] # Concentration of CO (Carbon monoxide), μg/m3
nh3 = [] # Concentration of NO (Nitrogen monoxide), μg/m3
no = [] # Сoncentration of NO2 (Nitrogen dioxide), μg/m3
no2 = [] # Сoncentration of O3 (Ozone), μg/m3
o3 = [] # Сoncentration of SO2 (Sulphur dioxide), μg/m3
pm10 = [] # Сoncentration of PM2.5 (Fine particles matter),
pm2_5 = [] # Сoncentration of PM10 (Coarse particulate matter), μg/m3
so2 = [] # Сoncentration of NH3 (Ammonia), μg/m3


for lt,ln in zip(lat,lon):
    response = requests.get(f'{air_url}lat={lt}&lon={ln}&appid={api_key}').json()
    #pprint (response)
    try:
        coord.append(response['coord'])
        aqi.append(response['list'][0]['main']['aqi'])
        co.append(response['list'][0]['components']['co'])
        nh3.append(response['list'][0]['components']['nh3'])
        no.append(response['list'][0]['components']['no'])
        no2.append(response['list'][0]['components']['no2'])
        o3.append(response['list'][0]['components']['o3'])
        pm10.append(response['list'][0]['components']['pm10'])
        pm2_5.append(response['list'][0]['components']['pm2_5'])
        so2.append(response['list'][0]['components']['so2'])
    except Exception as e:
        print (f'failed to get values for lat:{lt} and lon:{ln}. Error is: {e}')

In [9]:
response

{'coord': {'lon': -82.3101, 'lat': 34.7787},
 'list': [{'main': {'aqi': 2},
   'components': {'co': 236.99,
    'no': 0.04,
    'no2': 3.77,
    'o3': 80.11,
    'so2': 1.51,
    'pm2_5': 1.07,
    'pm10': 1.36,
    'nh3': 0.41},
   'dt': 1729118401}]}

In [10]:
# Create a DataFrame
map_data = pd.DataFrame({
    'Lat': lat,
    'Lon': lon,
    'AQI': aqi,
    'CO': co,
    'NH3': nh3,
    'NO2': no2,
    'O3': o3,
    'PM10': pm10,
    'PM2.5': pm2_5,
    'SO2': so2})



map_data.head()


,Lat,Lon,AQI,CO,NH3,NO2,O3,PM10,PM2.5,SO2
0,33.0890,-96.8864,2,230.31,0.92,4.16,87.26,2.19,1.70,0.47
1,20.2167,-100.8833,2,220.30,2.98,3.09,85.12,10.31,7.94,1.61
2,42.0406,-87.7826,2,363.83,1.66,45.24,34.69,7.49,4.75,2.09
3,42.0722,-87.7228,2,363.83,1.66,45.24,34.69,7.49,4.75,2.09
4,37.2296,-80.4139,2,210.29,0.13,3.90,70.81,1.54,1.24,3.46


In [11]:
map_data_copy = map_data.copy()
map_data_copy["Lat"] = map_data_copy["Lat"].astype(str)
map_data_copy["Lon"] = map_data_copy["Lon"].astype(str)

In [12]:
map_data_copy.dtypes

Lat       object
Lon       object
AQI        int64
CO       float64
NH3      float64
NO2      float64
O3       float64
PM10     float64
PM2.5    float64
SO2      float64
dtype: object

In [21]:
app = Dash(__name__)

In [22]:
# App layout
app.layout = html.Div([

    html.H1("Map of North America Showing Air Quality Data", style={'text-align': 'center'}),

    dcc.Dropdown(id="air_quality_selector",
                 options=[{'label': col, 'value': col} for col in map_data.columns[2:10]],
                 multi=False,
                 value=map_data.columns[2],
                 style={'width': "40%"},
                 ),

    html.Div(id='output_container', children=[]),
    html.Br(),

    dcc.Graph(id='aq_map', figure={})

])

In [23]:
# Connect the Plotly graphs with Dash Components
@app.callback(
    [Output(component_id='output_container', component_property='children'),
     Output(component_id='aq_map', component_property='figure')],
    [Input(component_id='air_quality_selector', component_property='value')]
)
def update_graph(chosen_airquality):

    container = "Displaying Air Quality Data for: {}".format(chosen_airquality)

    # map_data_copy = map_data.copy()
    # map_data_copy = map_data_copy[map_data_copy.columns.values[2:10] ==  chosen_airquality]

    # Plotly Express
    fig = px.scatter_mapbox(
        data_frame=map_data,
        lat= map_data['Lat'],
        lon= map_data['Lon'],
        color= map_data[chosen_airquality],
        hover_name=  map_data[chosen_airquality],
        color_continuous_scale=px.colors.sequential.YlOrRd,
        labels={"AQI": 'Scale'},
        mapbox_style= 'open-street-map'
    )

    return container, fig


In [24]:
# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)